# 引入

In [1]:
import torch
from re import L
from torch import nn
from torch.utils import data

# 数据

In [ ]:
def create_synthetic_data(w, b, num_examples):
  X = torch.normal(0, 1, (num_examples, len(w))) # 定义均值为0，方差为1，大小为样本数 x 每个样本的（这里可以是随机选）
  y = torch.matmul(X, w) + b
  y += torch.normal(0, 0.01, y.shape) # 加入噪音均值为0，方差为0.01（噪音一般用正态分布）
  return X, y.reshape((-1, 1)) # 把 y 变成列向量

true_w = torch.tensor([2, -3.4])
true_b = torch.tensor(4.2)
features, labels = create_synthetic_data(true_w, true_b, 1000)

def load_array(data_arrays, batch_size, is_train=True):
  # 这里的 * 的作用是把 (features, labels) 的括号拆掉
  dataset = data.TensorDataset(*data_arrays)
  return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 100
data_iter = load_array((features, labels), batch_size) # batch size 其实比较小是一件好事，因为越小噪音越多，反而避免一开始越走越歪

# 模型

In [ ]:
net = nn.Sequential(nn.Linear(2, 1))
net[0].weight.data.normal_(0, 0.01) # 初始化 w 和 b
net[0].bias.data.fill_(0)
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

# 训练

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
  for X, y in data_iter:
    l = loss(net(X), y)
    trainer.zero_grad()
    l.backward()
    trainer.step()

  l = loss(net(features), labels)
  print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.194035
epoch 2, loss 0.053535
epoch 3, loss 0.014853
epoch 4, loss 0.004195
epoch 5, loss 0.001240
epoch 6, loss 0.000418
epoch 7, loss 0.000188
epoch 8, loss 0.000124
epoch 9, loss 0.000106
epoch 10, loss 0.000101


能用牛顿法吗（二阶导）？不一定能求得到，就算可以，也要一个 nxn 的空间和计算，并不会很容易（一般只能做近似）